<a href="https://colab.research.google.com/github/akhilvydyula/Prediction_day_to_day_Prices/blob/master/making_day_to_day_pricing_decisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 brands and NAP in europe have 5 indivual maps they need day to day price decisions.

appr" 5k products name,brand,price for each product.simillar products like which is mirrror image to others



In [2]:
#import all the necessary packages.

from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")

In [11]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#lists the content of your google drive
!ls "/content/drive/My Drive/AAIC/"

 Automated-Response-Suggestion-for-Email-master.zip   Notes
 bow.csv					      qa_dataset.csv
 cleaned_strings				      resources.csv
 database.sqlite				      train.csv
 Db-IMDB.db					      train_data.csv
 deck.json					      training_text
 final_features.csv				      training_variants
 haberman.csv					      trainLabels.csv
 haberman.ipynb					      Train.zip
'Logistic Regression using SGD.ipynb'


In [0]:
#To open a file
import pandas as pd

# loading the data using pandas' read_json file.
data = pd.read_json('drive/My Drive/AAIC/deck.json',lines=True)

In [45]:
print ('Number of data points : ', data.shape[0], \
       '\nNumber of features/variables:', data.shape[1])

Number of data points :  4904 
Number of features/variables: 22


In [46]:
# each product/item has 22 features in the raw dataset.
data.columns # prints column-names or feature-names.

Index(['_id', 'name', 'sku', 'url', 'brand', 'media', 'meta', 'website_id',
       'price', 'description_text', 'spider', 'stock', 'classification',
       'created_at', 'updated_at', 'similar_products', 'positioning', 'lv_url',
       'price_changes', 'price_positioning', 'price_positioning_text',
       'sizes'],
      dtype='object')

In [47]:
print ('Number of data points : ', data.shape[0], \
       '\nNumber of features:', data.shape[1])

data.head(1) # prints the top rows in the table.

Number of data points :  4904 
Number of features: 22


,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes
0,{'$oid': '5da822166504fb65da005a59'},Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,"{'name': 'prada', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN


In [35]:
#description_text 
# We have total 72 unique type of product_type_names
print(data['description_text'].describe())

# 91.62% (167794/183138) of the products are shirts,


count              4904
unique             4764
top       Made in Italy
freq                  9
Name: description_text, dtype: object


In [36]:
# names of different product types
print(data['description_text'].unique())

['Gray and black silk-twill, black feathers Slips on  100% silk; trim: 100% feathers (Turkey, Ostrich) Dry clean  Feathers: South Africa  Made in Italy'
 'Black, white and gray gabardine Concealed zip fastening through front 100% polyester Dry clean  Made in Italy'
 'Black, red and cream silk crepe de chine Concealed button, hook and zip fastening at front  100% silk  Dry clean  Made in Italy'
 ...
 'Green and red Plexiglas®, gold-tone metal Push clasp fastening Comes in a presentation box Water resistant up to 30 meters Made in Switzerland'
 'Multicolored Plexiglas®, silver-tone zinc Push clasp fastening Comes in a presentation box Made in Italy'
 'Red, blue and baby-pink canvas, stainless steel Buckle fastening Comes in a presentation box Water resistant up to 50 meters Made in Switzerland']


In [38]:
# find the 10 most frequent product_type_names.
product_type_count = Counter(list(data['description_text']))

product_type_count.most_common(10)

[('Made in Italy', 9),
 ('Multicolored cotton-jersey Slips on 100% cotton Hand wash Made in Italy',
  6),
 ('', 6),
 ('Rubber sole measures approximately 20mm/ 1 inch Black and white stretch-knit Pull on Made in Italy',
  5),
 ('Clip fastening Made in Italy', 5),
 ('Heel measures approximately 30mm/ 1 inch Black leather Zip fastening along side',
  4),
 ('Multicolored cotton-jersey Slips on 100% cotton Machine wash', 4),
 ('Black cotton-twill 100% cotton Spot clean Made in Italy', 4),
 ('Multicolored linen Concealed zip fastening along back 100% linen; lining: 100% cotton Dry clean',
  4),
 ('Heel measures approximately 85mm/ 3.5 inches Black leather (Calf) Zip fastening along side Made in Italy',
  4)]

In [40]:
#basic statistics for brand feature
# there are 10577 unique brands
print(data['name'].describe())

# 183138 - 182987 = 151 missing values.

count                                                  4904
unique                                                 4578
top       Balenciaga  Speed stretch-knit high-top sneake...
freq                                                     11
Name: name, dtype: object


In [61]:
#basic statistics for brand feature
# there are 10577 unique brands
print(data['sizes'])

# 183138 - 182987 = 151 missing values.

8       [IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3...
49               [x small, small, medium, large, x large]
51      [IT35, IT35.5, IT36, IT36.5, IT37, IT37.5, IT3...
61      [IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3...
67               [x small, small, medium, large, x large]
                              ...                        
4843    [IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3...
4844    [IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3...
4851    [IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3...
4861    [IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3...
4883    [IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3...
Name: sizes, Length: 405, dtype: object


In [59]:
#basic statistics for brand feature
# there are 10577 unique brands
print(data['price_positioning'])

# 183138 - 182987 = 151 missing values.

8         0.0
49        0.0
51      100.0
61      100.0
67      100.0
        ...  
4843      0.0
4844      0.0
4851      0.0
4861      0.0
4883      0.0
Name: price_positioning, Length: 405, dtype: float64


In [66]:
#basic statistics for brand feature
# there are 10577 unique brands
print(data['brand'])

# 183138 - 182987 = 151 missing values.

8                     {'name': 'gucci', 'sub_brand': ''}
49                    {'name': 'spanx', 'sub_brand': ''}
51          {'name': 'stuart weitzman', 'sub_brand': ''}
61      {'name': 'christian louboutin', 'sub_brand': ''}
67                    {'name': 'spanx', 'sub_brand': ''}
                              ...                       
4843             {'name': 'jimmy choo', 'sub_brand': ''}
4844         {'name': 'gianvito rossi', 'sub_brand': ''}
4851                  {'name': 'gucci', 'sub_brand': ''}
4861                  {'name': 'gucci', 'sub_brand': ''}
4883    {'name': 'christian louboutin', 'sub_brand': ''}
Name: brand, Length: 405, dtype: object


In [53]:
# consider products which have price information
# data['formatted_price'].isnull() => gives the information 
#about the dataframe row's which have null values price == None|Null
data = data.loc[~data['price_positioning'].isnull()]
print('Number of data points After eliminating price=NULL :', data.shape[0])

Number of data points After eliminating price=NULL : 4901


In [55]:
# consider products which have color information
# data['color'].isnull() => gives the information about the dataframe row's which have null values price == None|Null
data =data.loc[~data['sizes'].isnull()]
print('Number of data points After eliminating color=NULL :', data.shape[0])

Number of data points After eliminating color=NULL : 405


In [62]:
# consider products which have color information
# data['color'].isnull() => gives the information about the dataframe row's which have null values price == None|Null
data =data.loc[~data['brand'].isnull()]
print('Number of data points After eliminating color=NULL :', data.shape[0])

Number of data points After eliminating color=NULL : 405


## removing duplicates

In [70]:
# find number of products that have duplicate titles.
print(sum(data.duplicated('name')))
# we have 2325 products which have same title but different color


8


In [74]:
# Remove All products with very few words in title
data_sorted = data[data['name'].apply(lambda x: len(x.split())>4)]
print("After removal of products with short description:", data_sorted.shape[0])

After removal of products with short description: 356


In [75]:
print(data['price'])

8       {'offer_price': {'currency': 'GBP', 'value': 4...
49      {'offer_price': {'currency': 'GBP', 'value': 3...
51      {'offer_price': {'currency': 'GBP', 'value': 3...
61      {'offer_price': {'currency': 'GBP', 'value': 9...
67      {'offer_price': {'currency': 'GBP', 'value': 8...
                              ...                        
4843    {'offer_price': {'currency': 'GBP', 'value': 3...
4844    {'offer_price': {'currency': 'GBP', 'value': 4...
4851    {'offer_price': {'currency': 'GBP', 'value': 5...
4861    {'offer_price': {'currency': 'GBP', 'value': 5...
4883    {'offer_price': {'currency': 'GBP', 'value': 5...
Name: price, Length: 405, dtype: object


In [77]:
# Sort the whole data based on title (alphabetical order of title) 
data_sorted.sort_values('name',inplace=True, ascending=False)
data_sorted.head(5)

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes
1272,{'$oid': '5db8119bd5424ee1f3353171'},adidas by Stella McCartney UltraBOOST X 3DS l...,1144205,https://www.net-a-porter.com/gb/en/product/114...,"{'name': 'adidas by stella mccartney', 'sub_br...","{'standard': [{'order': 1, 'url': 'https://cac...","{'breadcrumbs': {'1': 'shoes ', '2': ' sneaker...",{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 1...",Rubber sole measures approximately 30mm/ 1 inc...,netaporter_gb_pv,{'available': True},"{'l1': 'footwear', 'l2': 'shoes', 'l3': 'sneak...",{'$date': '2019-10-29T00:00:00.000Z'},{'$date': '2020-01-18T06:33:14.719Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 0, 'rank': 15}",,[{'date': {'$date': '2019-10-29T00:00:00.000Z'...,0.0,expensive,"[UK3.5, UK4, UK4.5, UK5, UK5.5, UK6, UK6.5, UK..."
1202,{'$oid': '5db81155d5424ee1f3352fc0'},adidas Originals Yung 1 nubuck and mesh sneak...,1141831,https://www.net-a-porter.com/gb/en/product/114...,"{'name': 'adidas originals', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'https://cac...","{'breadcrumbs': {'1': 'shoes ', '2': ' sneaker...",{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 2...","Sole measures approximately 25mm/ 1 inch Navy,...",netaporter_gb_pv,{'available': True},"{'l1': 'footwear', 'l2': 'shoes', 'l3': 'sneak...",{'$date': '2019-10-29T00:00:00.000Z'},{'$date': '2020-01-18T06:33:10.584Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 0, 'rank': 30}",,[{'date': {'$date': '2019-10-29T00:00:00.000Z'...,0.0,expensive,"[UK4, UK4.5, UK5, UK5.5, UK6, UK6.5, UK7, UK7...."
1203,{'$oid': '5db810ced5424ee1f3352b90'},adidas Originals EQT Gazelle rubber-trimmed l...,1141844,https://www.net-a-porter.com/gb/en/product/114...,"{'name': 'adidas originals', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'https://cac...","{'breadcrumbs': {'1': 'shoes ', '2': ' sneaker...",{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 3...",Rubber sole measures approximately 35mm/ 1.5 i...,netaporter_gb_pv,{'available': True},"{'l1': 'footwear', 'l2': 'shoes', 'l3': 'sneak...",{'$date': '2019-10-29T00:00:00.000Z'},{'$date': '2020-01-18T06:33:10.710Z'},"{'meta': {'total_results': 3, 'min_price': {'r...","{'page': 0, 'rank': 23}",,[{'date': {'$date': '2019-10-29T00:00:00.000Z'...,0.0,expensive,"[UK4, UK4.5, UK5, UK5.5, UK6, UK6.5, UK7, UK7...."
1204,{'$oid': '5db81155d5424ee1f3352fc6'},adidas Originals Continental 80 grosgrain-tri...,1141846,https://www.net-a-porter.com/gb/en/product/114...,"{'name': 'adidas originals', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'https://cac...","{'breadcrumbs': {'1': 'shoes ', '2': ' sneaker...",{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 3...",Rubber sole measures approximately 20mm/ 1 inc...,netaporter_gb_pv,{'available': True},"{'l1': 'footwear', 'l2': 'shoes', 'l3': 'sneak...",{'$date': '2019-10-29T00:00:00.000Z'},{'$date': '2020-01-18T06:33:09.934Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 0, 'rank': 37}",,[{'date': {'$date': '2019-10-29T00:00:00.000Z'...,0.0,expensive,"[UK4, UK4.5, UK5, UK5.5, UK6, UK6.5, UK7, UK7...."
302,{'$oid': '5dc459c92958d5a2a7b54be8'},adidas Originals Adibreak appliquéd striped s...,1096348,https://www.net-a-porter.com/gb/en/product/109...,"{'name': 'adidas originals', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'https://cac...",{'bert_original_classification': {'l1': 'cloth...,{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 5...","Black, white and light-gray satin-jersey Pull ...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'sportswear', 'l3': '...",{'$date': '2019-11-07T17:52:09.336Z'},{'$date': '

In [0]:
indices = []
for i,row in data_sorted.iterrows():
    indices.append(i)

In [82]:
print('Number of data points : ', data.shape[0])

Number of data points :  405


# text preprocessing

In [84]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [85]:
# we use the list of stop words that are downloaded from nltk lib.
stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalnum()))
            # Conver all letters to lower-case
            word = word.lower()
            # stop-word removal
            if not word in stop_words:
                string += word + " "
        data[column][index] = string

list of stop words: {'his', 'just', 'll', 'off', 'after', 'couldn', 'under', 'and', 'how', 'only', 'such', 'be', 'no', 'until', 'on', 'being', 'now', 'herself', 'why', 'own', "that'll", 'a', 'wasn', 'are', 'over', 'more', 'once', 'ain', 'here', 'few', 'further', 'not', 'yours', 'this', 'you', 'both', 'same', 'or', 'yourself', 'weren', 'out', 'm', 'wouldn', 'at', "she's", "won't", 'themselves', 'during', 'her', "mustn't", "it's", 'into', 'was', 'd', 'had', 'we', 't', 'but', 'i', 'didn', 'itself', 'yourselves', "couldn't", "mightn't", 'my', 'that', 'against', 'is', 'our', 'o', "isn't", 'there', 'again', 'below', "wouldn't", 'they', 'aren', 'have', 'than', 'them', 'some', 'those', 'if', 'as', 'the', 'very', 'won', 'himself', 'him', 'so', "you've", 'other', "hasn't", 'up', 'from', "you'll", 'mightn', 'will', "weren't", 'don', "shan't", 'hers', "aren't", 'been', 'when', 'about', 'their', 'myself', 'by', 'because', 'your', 'while', 'haven', 'did', 'y', "didn't", 'down', 'isn', 're', "shouldn

In [87]:
%%time
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in data.iterrows():
    nlp_preprocessing(row['name'], index, 'name')
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

0.11224299999999943 seconds
CPU times: user 113 ms, sys: 0 ns, total: 113 ms
Wall time: 117 ms


In [100]:
%%time
data.head(2)

CPU times: user 518 µs, sys: 979 µs, total: 1.5 ms
Wall time: 2.5 ms


,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes
8,{'$oid': '5db81088d5424ee1f33529d6'},gucci ace watersnaketrimmed embroidered leathe...,1009127,https://www.net-a-porter.com/gb/en/product/100...,"{'name': 'gucci', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'https://cac...","{'breadcrumbs': {'1': 'shoes ', '2': ' sneaker...",{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 4...",Sole measures approximately 20mm/ 1 inch White...,netaporter_gb_pv,{'available': True},"{'l1': 'footwear', 'l2': 'shoes', 'l3': 'sneak...",{'$date': '2019-10-29T00:00:00.000Z'},{'$date': '2020-01-18T06:10:47.605Z'},"{'meta': {'total_results': 3, 'min_price': {'r...","{'page': 5, 'rank': 14}",,[{'date': {'$date': '2019-10-29T00:00:00.000Z'...,0.0,expensive,"[IT34, IT34.5, IT35, IT35.5, IT36, IT36.5, IT3..."
49,{'$oid': '5db8110cd5424ee1f3352cb4'},spanx spotlight stretchtulle lace bodysuit,1050935,https://www.net-a-porter.com/gb/en/product/105...,"{'name': 'spanx', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'https://cac...","{'breadcrumbs': {'1': 'lingerie ', '2': ' shap...",{'$oid': '5da6d40110309200045874e6'},"{'offer_price': {'currency': 'GBP', 'value': 3...",Black stretch-tulle and lace Pulls on 72% nylo...,netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'laundry', 'l3': 'sha...",{'$date': '2019-10-29T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.348Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 11, 'rank': 47}",,[{'date': {'$date': '2019-10-29T00:00:00.000Z'...,0.0,expensive,"[x small, small, medium, large, x large]"


In [0]:
#Display an image
import PIL.Image
def display_img(url,ax,fig):
    # we get the url of the apparel and download it
    response = requests.get(url)
    img = PIL.Image.open(BytesIO(response.content))
    # we will display it in notebook 
    plt.imshow(img)
  
#plotting code to understand the algorithm's decision.
def plot_heatmap(keys, values, labels, url, text):
        # keys: list of words of recommended title
        # values: len(values) ==  len(keys), values(i) represents the occurence of the word keys(i)
        # labels: len(labels) == len(keys), the values of labels depends on the model we are using
                # if model == 'bag of words': labels(i) = values(i)
                # if model == 'tfidf weighted bag of words':labels(i) = tfidf(keys(i))
                # if model == 'idf weighted bag of words':labels(i) = idf(keys(i))
        # url : apparel's url

        # we will devide the whole figure into two parts
        gs = gridspec.GridSpec(2, 2, width_ratios=[4,1], height_ratios=[4,1]) 
        fig = plt.figure(figsize=(25,3))
        
        # 2nd, plotting image of the the apparel
        ax = plt.subplot(gs[1])
        # we don't want any grid lines for image and no labels on x-axis and y-axis
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
        
        # we call dispaly_img based with paramete url
        display_img(url, ax, fig)
        
        # displays combine figure ( heat map and image together)
        plt.show()
    


In [95]:
def plot_heatmap_image(doc_id, vec1, vec2, url, text, model):
    elif model == 'idf':
        labels = []
        for x in vec2.keys():
            # idf_title_vectorizer.vocabulary_ it contains all the words in the corpus
            # idf_title_features[doc_id, index_of_word_in_corpus] will give the idf value of word in given document (doc_id)
            if x in  idf_title_vectorizer.vocabulary_:
                labels.append(idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[x]])
            else:
                labels.append(0)
    plot_heatmap(keys, values, labels, url, text)


SyntaxError: ignored

In [0]:
# this function gets a list of wrods along with the frequency of each 
# word given "text"
def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    # words stores list of all words in given string, you can try 'words = text.split()' this will also gives same result
    return Counter(words) # Counter counts the occurence of each word in list, it returns dict type object {word1:count}


def get_result(doc_id, content_a, content_b, url, model):
    text1 = content_a
    text2 = content_b
    
    # vector1 = dict{word11:#count, word12:#count, etc.}
    vector1 = text_to_vector(text1)

    # vector1 = dict{word21:#count, word22:#count, etc.}
    vector2 = text_to_vector(text2)

    plot_heatmap_image(doc_id, vector1, vector2, url, text2, model)

## extracting name features

In [0]:
idf_title_vectorizer = CountVectorizer()
idf_title_features = idf_title_vectorizer.fit_transform(data['name'])

# idf_title_features.shape = #data_points * #words_in_corpus
# CountVectorizer().fit_transform(courpus) returns the a sparase matrix of dimensions #data_points * #words_in_corpus
# idf_title_features[doc_id, index_of_word_in_corpus] = number of times the word occured in that doc

In [0]:
def n_containing(word):
    # return the number of documents which had the given word
    return sum(1 for blob in data['name'] if word in blob.split())

def idf(word):
    # idf = log(#number of docs / #number of docs which had the given word)
    return math.log(data.shape[0] / (n_containing(word)))

In [0]:
# we need to convert the values into float
idf_title_features  = idf_title_features.astype(np.float)

for i in idf_title_vectorizer.vocabulary_.keys():
    # for every word in whole corpus we will find its idf value
    idf_val = idf(i)
    
    # to calculate idf_title_features we need to replace the count values with the idf values of the word
    # idf_title_features[:, idf_title_vectorizer.vocabulary_[i]].nonzero()[0] will return all documents in which the word i present
    for j in idf_title_features[:, idf_title_vectorizer.vocabulary_[i]].nonzero()[0]:
        
        # we replace the count values of word i in document j with  idf_value of word i 
        # idf_title_features[doc_id, index_of_word_in_courpus] = idf value of word
        idf_title_features[j,idf_title_vectorizer.vocabulary_[i]] = idf_val
        

In [107]:
# some of the brand values are empty. 
# Need to replace Null with string "NULL"
data['brand'].fillna(value="Not given", inplace=True )

# replace spaces with hypen
brands = [x.replace(" ", "-") for x in data['brand'].values]

brand_vectorizer = CountVectorizer()
brand_features = brand_vectorizer.fit_transform(brands)



extra_features = hstack((brand_features)).tocsr()

AttributeError: ignored